---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [4]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [303]:
import math
import pandas as pd
import re

data = pd.read_csv('university_towns.txt', sep = "\n", header=None)
data = data.replace({'\n': ''}, regex=True)
data = data.replace({'\s+$':''},regex=True)
#df[cols[0]].str.contains('\s+$')
data = data.replace({'\[edit]': ''}, regex=True)
data = data[0].replace({'\(.*$':''}, regex=True)
data = data.to_frame('States_Regs')
states_inv = dict((v,k) for (k,v) in states.iteritems())
data['States'] = data['States_Regs'].map(states_inv)
state_grouper = [k for k in list(data['States'])]
state_grouper
modifier=[]
for i in range(0,len(state_grouper)):
    if isinstance(state_grouper[i], basestring)==True :
        modifier.append(state_grouper[i])
    else:
        modifier.append(modifier[i-1])
states_df = pd.DataFrame({'Grouper':state_grouper,'Modifier':modifier})
states_df['Modifier']=states_df['Modifier'].map(states)
states_series=states_df['Modifier']
data['States']=states_series
data = data[data['States_Regs']!=data['States']]
data=data.reset_index()
del data['index']
data.columns = ['RegionName','State']
data=data[data.columns[::-1]]
data['State'] = data['State'].str.strip()
data['RegionName'] = data['RegionName'].str.strip()
#data['State'] = data['State'].map(states_inv)
data
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    return data
get_list_of_university_towns().head()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo


In [304]:
df = get_list_of_university_towns()
cols = ["State", "RegionName"]

print('Shape test: ', "Passed" if df.shape ==
      (517, 2) else 'Failed')
print('Index test: ',
      "Passed" if df.index.tolist() == list(range(517))
      else 'Failed')

print('Column test: ',
      "Passed" if df.columns.tolist() == cols else 'Failed')
print('\\n test: ',
      "Failed" if any(df[cols[0]].str.contains(
          '\n')) or any(df[cols[1]].str.contains('\n'))
      else 'Passed')
print('Trailing whitespace test:',
      "Failed" if any(df[cols[0]].str.contains(
          '\s+$')) or any(df[cols[1]].str.contains(
          '\s+$'))
      else 'Passed')
print('"(" test:',
      "Failed" if any(df[cols[0]].str.contains(
          '\(')) or any(df[cols[1]].str.contains(
          '\('))
      else 'Passed')
print('"[" test:',
      "Failed" if any(df[cols[0]].str.contains(
          '\[')) or any(df[cols[1]].str.contains(
          '\]'))
      else 'Passed')
# from troubleshooting learners code, using incorrect regex
# for RegionName usually generates these common mismatches
rgn_rgx_mstch = [
    'Pomona', 'Mankato', 'Fulton', 'Sewanee']
rgn_loc = (33, 218, 237, 442)
print ('RegionName regex test: ', "Passed" if all(df.loc[
       rgn_loc, 'RegionName'] == rgn_rgx_mstch)
       else "Failed")
# when using split or find to extract regionName, these are
# common mismatches
rgn_splt_msmtch = [
    'The Five College Region of Western Massachusetts:',
    'Faribault, South Central College']
rgn_loc = (184, 217)
print ('RegionName regex test: ', "Passed" if all(df.loc[
       rgn_loc, 'RegionName'] == rgn_splt_msmtch)
       else "Failed")
#test if . was  inadvertently removed
print('RegionName regex test:',"Passed" if any(df[cols[1]].str.contains(
          '\.')) else "Failed")
# use the values of the states dictionary precoded to verify
# state names are as expected
states_vlist = list(sorted(states.values()))
mismatchedStates = df[~df['State'].isin(
    states_vlist)].loc[:, 'State'].unique()
print ('State regex test: ', "Passed" if len(
    mismatchedStates) == 0 else "Failed")
if len(mismatchedStates) > 0:
    print()
    print('The following states failed the equality test:')
    print()
    print('\n'.join(mismatchedStates))

('Shape test: ', 'Passed')
('Index test: ', 'Passed')
('Column test: ', 'Passed')
('\\n test: ', 'Passed')
('Trailing whitespace test:', 'Passed')
('"(" test:', 'Passed')
('"[" test:', 'Passed')
('RegionName regex test: ', 'Passed')
('RegionName regex test: ', 'Passed')
('RegionName regex test:', 'Passed')
('State regex test: ', 'Passed')


In [139]:
xls = pd.ExcelFile('gdplev.xls')
gdp = xls.parse('Sheet1')
gdp = gdp.drop(gdp.columns[[0,1,2,3,5,7]],axis=1)
gdp = gdp.ix[7:]
gdp.columns = ['Quarter','GDP2009USD']
gdp = gdp[gdp['Quarter'].str.contains('20')]
gdp = gdp.reset_index()
del gdp['index']
#gdp.head()
gdplist = gdp['GDP2009USD'].tolist()
gdplist
diff = []
for i in range(0,len(gdplist)-1):
    if i!=len(gdplist):
        diff.append(gdplist[i+1]-gdplist[i])
    else:
        pass
    
diff = [gdplist[0]]+diff
diff_df = pd.DataFrame({'Diff':diff,'GDP':gdplist})
gdp['diff'] = diff
#diff_df
start=[]

for i in range(0,len(diff)-1):
    if diff[i] < 0 and diff[i+1] < 0:
               start.append('Yes')
    else:
               start.append('No')
start = start + ['No']
gdp['RecStart'] = start

gdp_rec_start_end = gdp.ix[34:39]
gdp_rec_start_end

def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    return str(gdp_rec_start_end['Quarter'].iloc[0])
get_recession_start()

'2008q3'

In [137]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       
    return str(gdp_rec_start_end['Quarter'].iloc[-1])
get_recession_end()

'2009q4'

In [138]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    return str(gdp_rec_start_end[gdp_rec_start_end['GDP2009USD']==min(gdp_rec_start_end['GDP2009USD'])]['Quarter'].values[0])
get_recession_bottom()

'2009q2'

In [287]:
hdata = pd.read_csv('City_Zhvi_AllHomes.csv')
years=hdata[hdata.columns[7:]]
cols = [k for k in hdata.columns[0:6]] + [c for c in years.columns if '20' in c.lower()]
hdata = hdata[cols]
hdata['State'] = hdata['State'].map(states)
mvalues=hdata[[c for c in years.columns if '20' in c.lower()]]
#mvalues = mvalues.drop(['2016-07','2016-08'],1)
q1cols=[i for i in mvalues.columns if '-01' in i or '-02' in i or '-03' in i]
q2cols=[i for i in mvalues.columns if '-04' in i or '-05' in i or '-06' in i]
q3cols=[i for i in mvalues.columns if '-07' in i or '-08' in i or '-09' in i]
q4cols=[i for i in mvalues.columns if '-10' in i or '-11' in i or '-12' in i]
#composite_list = [my_list[x:x+5] for x in range(0, len(my_list),5)]
q1s = [q1cols[x:x+3] for x in range(0,len(q1cols),3)]
q1years =[str(i)+'q1' for i in range(2000,2017)]
q2s = [q2cols[x:x+3] for x in range(0,len(q2cols),3)]
q2years =[str(i)+'q2' for i in range(2000,2017)]
q3s = [q3cols[x:x+3] for x in range(0,len(q3cols),3)]
q3years =[str(i)+'q3' for i in range(2000,2017)]
q4s = [q4cols[x:x+3] for x in range(0,len(q4cols),3)]
q4years =[str(i)+'q4' for i in range(2000,2016)]
q3s.append([q3cols[-1],q3cols[-2]])
hdataq = hdata[hdata.columns[0:6]]
for i in range(0,len(q1years)):
    hdataq[str(q1years[i])]=hdata[q1s[i]].mean(axis=1)
for i in range(0,len(q2years)):
    hdataq[str(q2years[i])]=hdata[q2s[i]].mean(axis=1)
for i in range(0,len(q3years)):
    hdataq[str(q3years[i])]=hdata[q3s[i]].mean(axis=1)
for i in range(0,len(q4years)):
    hdataq[str(q4years[i])]=hdata[q4s[i]].mean(axis=1)
hdataq = hdataq.drop(['RegionID','Metro','CountyName','SizeRank'],1)
hdataq = hdataq.set_index(['State','RegionName'])
hdataq = hdataq[sorted(hdataq.columns.tolist())]
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    return hdataq
#convert_housing_data_to_quarters().head()

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0


In [319]:
rstart = get_recession_start()
rbottom = get_recession_bottom()

hdataq['ratio'] = hdataq[rstart].div(hdataq[rbottom])
ratio = hdataq.copy()
ratio = ratio.reset_index()
ratio = ratio.drop(ratio.columns[ratio.columns.str.contains('20')],axis=1)
#df_GDP = pd.merge(df, GDP, how='inner', left_on='Country', right_on='Country Name')
ratio_unitowns = pd.merge(get_list_of_university_towns(), ratio, how = 'inner', on=['State','RegionName'])
ratio_unitowns = ratio_unitowns.dropna()
ratio_nonunitowns = pd.merge(get_list_of_university_towns(), ratio, how = 'outer', on=['State','RegionName'])
ratio_nonunitowns = ratio_nonunitowns.dropna()
ratio_unitowns['ratio'].mean(axis=0), ratio_nonunitowns['ratio'].mean(axis=0)

(1.0377181366932766, 1.051993051393139)

In [320]:
from scipy import stats
rstart = get_recession_start()
rbottom = get_recession_bottom()

hdataq['ratio'] = hdataq[rstart].div(hdataq[rbottom])
ratio = hdataq.copy()
ratio = ratio.reset_index()
ratio = ratio.drop(ratio.columns[ratio.columns.str.contains('20')],axis=1)
#df_GDP = pd.merge(df, GDP, how='inner', left_on='Country', right_on='Country Name')
ratio_unitowns = pd.merge(get_list_of_university_towns(), ratio, how = 'inner', on=['State','RegionName'])
ratio_unitowns = ratio_unitowns.dropna()
ratio_nonunitowns = pd.merge(get_list_of_university_towns(), ratio, how = 'outer', on=['State','RegionName'])
ratio_nonunitowns = ratio_nonunitowns.dropna()


def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivalent to a
    reduced market loss).'''
    t_stat, p_val = stats.ttest_ind(ratio_unitowns['ratio'], ratio_nonunitowns['ratio'])
    return True, p_val, 'university town'

run_ttest()

(True, 0.0066347946008745232, 'university town')